In [1]:
import numpy as np
import cv2 as cv2
import datetime
from cv2 import aruco

In [ ]:
# Define ARUCO dictionary to detect
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

#Initialize the video capture (0 for computer camera)
cap = cv2.VideoCapture(1)

while True:
    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame")
        break

    # Convert frame to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    if ids is not None and len(ids) >= 4:  # Ensure at least 4 markers are detected
        # Draw detected markers for visualization
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        # Collect the corners of the 4 markers
        marker_corners = {}
        for i, marker_id in enumerate(ids.flatten()):
            marker_corners[marker_id] = corners[i][0]  # Extract the corners for each detected marker

        # Check if specific 4 marker IDs are detected (e.g., IDs 0, 1, 2, 3)
        required_ids = [0, 1, 2, 3]
        if all(marker_id in marker_corners for marker_id in required_ids):
            # Get the centers of the 4 markers
            centers = {marker_id: np.mean(marker_corners[marker_id], axis=0) for marker_id in required_ids}

            # Sort the points for consistent perspective correction
            # Top-left, top-right, bottom-right, bottom-left order
            sorted_ids = sorted(centers.items(), key=lambda x: (x[1][1], x[1][0]))  # Sort by Y, then by X
            sorted_points = [x[1] for x in sorted_ids]  # Extract the points in sorted order

            # Create source and destination points for perspective transform
            src_points = np.array(sorted_points, dtype="float32")
            dst_points = np.array([[0, 0], [500, 0], [500, 500], [0, 500]], dtype="float32")

            # Compute the perspective transform matrix
            matrix = cv2.getPerspectiveTransform(src_points, dst_points)

            # Warp the perspective to get the focused rectangle
            warped = cv2.warpPerspective(frame, matrix, (500, 500))

            # Display the warped image (focused rectangle)
            cv2.imshow("Warped View", warped)

    # Display the original frame with detected markers
    cv2.imshow("Original Frame", frame)

    

    



In [43]:
def image_smoothing(img, sc = 1000, ss = 3000, diameter = 20):
    smooth_img = cv2.bilateralFilter(img, d=diameter, sigmaColor = sc, sigmaSpace = ss)
    return smooth_img

def image_hsv(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return img_hsv

def get_obstacles(frame):
    img_copy = frame.copy()

    #1) Convert to HSV
    img_hsv = image_hsv(img_copy)

    #2) Smooth image to remove noise and smoothen the gradiant
    img_smooth = image_smoothing(img_hsv)

    #3) Mask for obstacles
    #obstaces colour (black). Colour red when detected
    obst_bound = np.array([[0, 0, 0], [180,220,90], [0, 0 , 255]])
    min_bound = np.array(obst_bound[0], np.uint8)
    max_bound = np.array(obst_bound[1], np.uint8)
    img_masked = cv2.inRange(img_smooth, min_bound, max_bound)
    

    #4) obstacle detection
    contours, _ = cv2.findContours(img_masked, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    return contours


image_path = "output_frame.jpg" 

#cap = cv2.VideoCapture(0)
#ret, frame = cap.read()

frame = cv2.imread(image_path)
contours = get_obstacles(frame)
# Display the result
cv2.drawContours(frame, contours, -1, (0, 255, 0), 1)
cv2.imshow("Corrected Rectangle", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()
